In [3]:
DataSource('instruments_CN_FUND').read().head()

,date,instrument,name
0,2005-01-04,510050.OFA,华夏上证50ETF
1,2005-01-05,510050.OFA,华夏上证50ETF
2,2005-01-06,510050.OFA,华夏上证50ETF
3,2005-01-07,510050.OFA,华夏上证50ETF
4,2005-01-10,510050.OFA,华夏上证50ETF


In [4]:
df = DataSource('basic_info_CN_FUND').read()  # .read(instrument='510300.HOF', start_date='2017-1-1').head()  # 
df[df['instrument']=='510300.HOF']

,abbreviation_name,delist_date,display_name,instrument,list_date,name,type
1072,NaN,2200-01-01,300ETF,510300.HOF,2012-05-28,300ETF,etf


In [18]:
m7.data.read().head(3)

,adjust_factor,amount,close,date,high,instrument,low,open,paused,turn,volume,"mean(close,20)","mean(close,60)"
200,1.013924,1.012514e+09,2.467,2013-04-18,2.486,510300.HOF,2.438,2.439,0.0,0.063927,411305194.0,2.51080,2.587783
201,0.382228,1.266655e+08,2.463,2013-04-18,2.476,159919.ZOF,2.431,2.446,0.0,0.003891,51543848.0,2.50555,2.583583
202,1.141400,7.082894e+08,1.766,2013-04-18,1.774,510050.HOF,1.742,1.750,0.0,0.044012,402712905.0,1.81165,1.885900


In [3]:
# 本代码由可视化策略环境自动生成 2021年8月22日10:08
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m2_run_bigquant_run(input_1, start_date,end_date):
    # 示例代码如下。在这里编写您的代码
    ins=input_1.read_pickle()['instruments']
    start_date=input_1.read_pickle()['start_date']
    end_date=input_1.read_pickle()['end_date']
    df=DataSource('bar1d_CN_FUND').read(ins,start_date=start_date,end_date=end_date)
    data_1 = DataSource.write_df(df)
    return Outputs(data_1=data_1, data_2=None, data_3=None)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m2_post_run_bigquant_run(outputs):
    return outputs

# 回测引擎：初始化函数，只执行一次
def m4_initialize_bigquant_run(context):

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

# 回测引擎：每日数据处理函数，每天执行一次
def m4_handle_data_bigquant_run(context, data):
    # 获取今日的日期
    today = data.current_dt.strftime('%Y-%m-%d')  
    data_today=context.data[context.data['date']==today]
    sid = context.symbol(context.instruments[0]) 
    price = data.current(sid, 'price')
    cash = context.portfolio.cash 
    #cur_position = context.portfolio.positions[sid].amount

    short_avg=np.array(data_today['mean(close,20)'])[0]
    long_avg=np.array(data_today['mean(close,60)'])[0]
    
    
    if short_avg>long_avg :
        context.order(sid,int(cash/price/100)*100)
    if short_avg<long_avg :
        context.order_target_percent(sid, 0)
# 回测引擎：准备数据，只执行一次
def m4_prepare_bigquant_run(context):
    # 加载预测数据
    
    df = context.options['data'].read_df()
    context.data=df
    pass
# 回测引擎：每个单位时间开始前调用一次，即每日开盘前调用一次。
def m4_before_trading_start_bigquant_run(context, data):
    pass

m3 = M.input_features.v1(
    features="""
# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征

mean(close,20)
mean(close,60)
""",
    m_cached=False
)

m1 = M.instruments.v2(
    start_date='2018-08-29',
    end_date='2021-09-01',
    market='CN_FUND',
    instrument_list='',
    max_count=0
)

m2 = M.cached.v3(
    input_1=m1.data,
    run=m2_run_bigquant_run,
    post_run=m2_post_run_bigquant_run,
    input_ports='',
    params='{\'start\':\'2018-08-29\'}',
    output_ports=''
)

m5 = M.derived_feature_extractor.v3(
    input_data=m2.data_1,
    features=m3.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=True,
    remove_extra_columns=False,
    user_functions={}
)

m7 = M.dropnan.v2(
    input_data=m5.data
)

m4 = M.trade.v4(
    instruments=m1.data,
    options_data=m7.data,
    history_ds=m7.data,
    start_date='2020-1-1',
    end_date='',
    initialize=m4_initialize_bigquant_run,
    handle_data=m4_handle_data_bigquant_run,
    prepare=m4_prepare_bigquant_run,
    before_trading_start=m4_before_trading_start_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='close',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark='510300.HOF'
)


[2021-08-22 10:08:32.370713] INFO: moduleinvoker: input_features.v1 开始运行..

[2021-08-22 10:08:32.413332] INFO: moduleinvoker: input_features.v1 运行完成[0.042633s].

[2021-08-22 10:08:32.421341] INFO: moduleinvoker: instruments.v2 开始运行..

[2021-08-22 10:08:32.627406] INFO: moduleinvoker: instruments.v2 运行完成[0.206058s].

[2021-08-22 10:08:32.642850] INFO: moduleinvoker: cached.v3 开始运行..

...

[2021-08-22 10:08:40.084246] INFO: moduleinvoker: derived_feature_extractor.v3 运行完成[4.41896s].

[2021-08-22 10:08:40.095919] INFO: moduleinvoker: dropnan.v2 开始运行..

[2021-08-22 10:08:40.132751] INFO: dropnan: /data, 0/0

[2021-08-22 10:08:40.159392] ERROR: moduleinvoker: module name: dropnan, module version: v2, trackeback: Exception: no data left after dropnan\n

Exception: no data left after dropnan